In [69]:
from matplotlib import pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
import pandas as pd
from solvers.catchment_runoff_solver import CatchmentRunoffSolver

storm_rainfall_depth = pd.read_csv("data/PF_Depth_English_PDS.csv")

catchment_area=20.4
impervious_cover_fraction=0.23

storm_depths = []
for index, row in storm_rainfall_depth.iterrows():
    for ri in list(storm_rainfall_depth.drop(["Storm"], axis=1).columns):
        depth = row[ri]
        
        rv = CatchmentRunoffSolver(
            catchment_area=catchment_area,
            rainfall_depth=depth,
            impervious_cover_fraction=impervious_cover_fraction
        ).solve()
        
        length = row["Storm"].replace(':', '')
        
        storm_depths.append([ri, length, depth, rv])

storm_depths_df = pd.DataFrame(storm_depths, columns=['recurrence interval (yr)', 'storm length (min)', 'rainfall depth (in)', 'runoff volume (ft^3)'])

# fig, ax = plt.subplots(figsize=(12, 8))
# 
# p1, = ax.plot(storm_depths_df["rainfall depth (in)"], storm_depths_df['runoff volume (ft^3)'], color='blue', marker='o',
#               label="Runoff Volume")
# 
# ax.legend(handles=[p1])

# plt.title(title)
storm_depths_df


,recurrence interval (yr),storm length (min),rainfall depth (in),runoff volume (ft^3)
0,1,5-min,0.431,8202.52
1,2,5-min,0.524,9972.43
2,5,5-min,0.668,12712.95
3,10,5-min,0.795,15129.93
4,25,5-min,0.977,18593.64
...,...,...,...,...
185,50,60-day,24.300,462462.15
186,100,60-day,26.400,502428.01
187,200,60-day,28.500,542393.87
188,500,60-day,31.200,593778.56


In [71]:
from solvers import ChannelType
from solvers.channel_discharge_solver import ChannelDischargeSolver
from solvers.volume_solver import VolumeSolver

q_a_solve = ChannelDischargeSolver(
    channel_type=ChannelType.Trapezoid,
    manning_roughness_coefficient=0.09,
    channel_slope=0.2,
    bottom_width_channel=1.513888889,
    depth_flow=0.805555556,
    side_slope=0.032754349
)

q_a = q_a_solve.solve()

v_a = VolumeSolver(
    channel_type=ChannelType.Trapezoid,
    channel_length=160,
    bottom_width_channel=1.513888889,
    top_width_channel=4.479166667,
    depth_flow=0.805555556
).solve()

q_c_solve = ChannelDischargeSolver(
    channel_type=ChannelType.Trapezoid,
    manning_roughness_coefficient=0.04,
    channel_slope=0.2,
    bottom_width_channel=1.5625,
    depth_flow=1.3125,
    side_slope=0.014176131
)
q_c = q_c_solve.solve()
v_c = VolumeSolver(
    channel_type=ChannelType.Trapezoid,
    channel_length=58,
    bottom_width_channel=1.5625,
    top_width_channel=4.416666667,
    depth_flow=1.3125
).solve()

v_b1 = VolumeSolver(
    channel_type=ChannelType.Rectangle,
    channel_length=19,
    top_width_channel=7.354166667,
    depth_flow=0.805555556
).solve()

v_b2 = VolumeSolver(
    channel_type=ChannelType.Rectangle,
    channel_length=14,
    top_width_channel=4.861111111,
    depth_flow=1.111111111
).solve()

v_c_summed = round(sum([v_c, v_b1, v_b2]), 2)

swale_capacity = [
    ['A', q_a, v_a, round(v_a / q_a, 2)], 
    ['C', q_c, v_c, round(v_c / q_c, 2)],
    ['C w/ Basins', q_c, v_c_summed, round(v_c_summed / q_c, 2)]
]

swale_capacity_df = pd.DataFrame(swale_capacity, columns=['swale name', 'peak discharge capacity (ft^3/s)', 'volume capacity (ft^3)', 'time to fill (s)'])
swale_capacity_df

,swale name,peak discharge capacity (ft^3/s),volume capacity (ft^3),time to fill (s)
0,A,4.96,386.22,77.87
1,C,21.65,227.58,10.51
2,C w/ Basins,21.65,415.76,19.20


In [72]:
from solvers.ponding_soil_infiltration_solver import PondingSoilInfiltrationSolver

a_s = PondingSoilInfiltrationSolver(
    wetted_perimeter = q_a_solve.wp,
    flow_volume = v_a,
    channel_length = 160,
    soil_infiltration_rate = 0.4,
    soil_depth = 6.56168,
    soil_water_capacity = 0.14
).solve()

c_s = PondingSoilInfiltrationSolver(
    wetted_perimeter = q_c_solve.wp,
    flow_volume = v_c,
    channel_length = 58,
    soil_infiltration_rate = 0.2,
    soil_depth = 6.56168,
    soil_water_capacity = 0.14
).solve()

c_ss = PondingSoilInfiltrationSolver(
    wetted_perimeter = q_c_solve.wp,
    flow_volume = v_c_summed,
    channel_length = 58,
    soil_infiltration_rate = 0.2,
    soil_depth = 6.56168,
    soil_water_capacity = 0.14
).solve()

swale_infiltration = [
    ['A', a_s],
    ['C', c_s],
    ['C w/ Basins', c_ss]
]

swale_infiltration_df = pd.DataFrame(swale_infiltration, columns=['swale name', 'infiltration time (hr)'])
swale_infiltration_df

,swale name,infiltration time (hr)
0,A,23.17
1,C,56.22
2,C w/ Basins,102.70
